In [106]:
import pandas as pd
import numpy as np
import os 
from typing import Dict, Any


def convert_1m_csv_to_5m_df(file_path: str) -> pd.DataFrame:
    # Column Names (သင့်ရဲ့ Data အစီအစဉ်အတိုင်း)
    COLUMN_NAMES = ['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    DATETIME_FORMAT = '%Y.%m.%d %H:%M'

    """ CSV File မှ 1-Minute Data ကို Load ပြီး 5-Minute Candle သို့ ပြောင်းလဲသည်။ """
    if not os.path.exists(file_path):
        print(f"🚨 Error: File not found at path: {file_path}")
        return pd.DataFrame()

    try:
        df = pd.read_csv(file_path, sep=',', header=None, names=COLUMN_NAMES,
                         dtype={'Open': np.float64, 'High': np.float64, 'Low': np.float64, 'Close': np.float64})
        
        # Volume column ကို ယာယီ float အနေဖြင့် သတ်မှတ်ပြီး NA များကို 0 ဖြင့် အစားထိုးသည်။
        df['Volume'] = pd.to_numeric(df['Volume'], errors='coerce').fillna(0)
        df['Volume'] = df['Volume'].astype(np.int64) 
        
    except Exception as e:
        print(f"🚨 Error loading CSV file: {e}")
        return pd.DataFrame()

    # Datetime Index ကို တည်ဆောက်ခြင်း
    df['Datetime'] = df['Date'].astype(str) + ' ' + df['Time'].astype(str)
    df['Datetime'] = pd.to_datetime(df['Datetime'], format=DATETIME_FORMAT, errors='coerce')
    df.set_index('Datetime', inplace=True)
    df.drop(columns=['Date', 'Time'], inplace=True)
    #df.dropna(subset=[df.index.name], inplace=True) # Invalid Datetime များကို ဖယ်ရှားခြင်း

    ohlcv_aggregation_rules: Dict[str, Any] = {
        'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'
    }
    df_5m = df.resample('5Min').agg(ohlcv_aggregation_rules)
    df_5m.dropna(inplace=True)
    df_5m = df_5m[df_5m['Volume'] > 0]
    
    print(f"✅ Conversion successful! 5-Min rows: {len(df_5m)}")
    return df_5m

In [107]:
import json

class EnvConfig():
    """environment configuration from json file
       tgym requires you configure your own parameters in json file.
        Args:
            config_file path/file.json

    """
    def __init__(self,config_file):
        self.config = {}
        with open(config_file) as j:
            self.config = json.load(j)

    def env_parameters(self,item=''):
        """environment variables
        """
        if item:
            return self.config["env"][item]
        else:
            return self.config["env"]

    def symbol(self, asset="GBPUSD", item='') :
        """get trading pair (symbol) information

        Args:
            asset (str, optional): symbol in config. Defaults to "GBPUSD".
            item (str, optional): name of item, if '' return dict, else return item value. Defaults to ''.

        Returns:
            [type]: [description]
        """
        if item:
            return self.config["symbol"][asset][item]
        else:
            return self.config["symbol"][asset]

    def data_processing_parameters(self, item=''):
        """Get data processing config"""
        if item:
            return self.config["data_processing"][item]
        return self.config["data_processing"]

    def trading_hour(self,place="NewYork"):
        """forex trading hour from different markets

        Args:
            place (str, optional): [Sydney,Tokyo,London] Defaults to "New York".

        Returns:
            [dict]: from time, to time
        """
        if place:
            return self.config["trading_hour"][place]
        else:
            return self.config["trading_hour"]
        
    def indicator(self,place="sma_fast_period"):
        """forex trading hour from different markets

        Args:
            place (str, optional): [Sydney,Tokyo,London] Defaults to "New York".

        Returns:
            [dict]: from time, to time
        """
        if place:
            return self.config["data_processing"]["indicator"][place]
        else:
            return self.config["data_processing"]["indicator"]



In [108]:

import pandas as pd
import logging
from finta import TA
from sklearn.preprocessing import StandardScaler
import holidays
import os
import json

import logging
# Configure logging
logger = logging.getLogger(__name__)

def patch_missing_data(df, dt_col_name='time', cf=None):
    min_bars = cf.data_processing_parameters("min_bars_per_week")

    # ["time","open", "high", "low", "close"]
    required_cols = cf.data_processing_parameters("required_cols")

    # df မှာ 6 columns ရှိရင် vol ပါထည့်မယ်
    if df.shape[1] == 6:
        df.columns = required_cols + ['vol']
    elif df.shape[1] == 5:
        df.columns = required_cols
    else:
        raise ValueError(f"Invalid number of columns: {df.shape[1]} =>{required_cols}")

    logger.warning(f"shape of  column: {df.shape[1]}")
    # 1. Column validation
    if missing := set(required_cols) - set(df.columns):
        raise ValueError(f"Missing columns: {missing}")

    # 2. Auto-detect datetime column
    dt_candidates = {'time', 'timestamp', 'date', 'datetime'}
    if dt_col_name not in df.columns:
        found = list(dt_candidates & set(df.columns))
        if not found:
            raise KeyError(f"No datetime column found. Tried: {dt_candidates}")
        dt_col_name = found[0]
        logger.info(f"Using datetime column: {dt_col_name}")

    # 3. Convert to datetime index
    df[dt_col_name] = pd.to_datetime(df[dt_col_name], utc=True)
    df = df.set_index(dt_col_name).sort_index()
    groups = df.groupby(pd.Grouper(freq='W-SUN'))

    patched_weeks = []  # patched weekly df storage

    for w, week_df in groups:
        if week_df.empty:
            continue

        if len(week_df) != min_bars:
            logger.warning(f"Week {w} has {len(week_df)}/{min_bars} bars")

        # Create 5-minute frequency index
        new_index = pd.date_range(
            start=week_df.index.min(),
            end=week_df.index.max(),
            freq='5min',
            tz='UTC'
        )

        # Reindex + forward fill
        week_df = week_df.reindex(new_index)
        week_df.index = week_df.index.tz_localize(None)
        fill_limit = 12 # ဥပမာ: 1 နာရီ (12 bars) ထက်ပိုတဲ့ ကွက်လပ်ကို မဖြည့်ပါ
        fill_cols = ['open', 'high', 'low', 'close', 'vol'] if 'vol' in df.columns else ['open', 'high', 'low', 'close']
        # FFill: ရှေ့က data ဖြင့် ဖြည့်ပါ
        week_df[fill_cols] = week_df[fill_cols].ffill(limit=fill_limit)
        patched_weeks.append(week_df)

    # Merge back all weeks
    if patched_weeks:
        all_df = pd.concat(patched_weeks)
    else:
        all_df = df.copy()

    return all_df

In [109]:

def add_time_feature(df_5m: pd.DataFrame, cf=None, source_tz='UTC') -> pd.DataFrame:
    """
    5M Data Frame (DatetimeIndex ပါဝင်သည်ဟု ယူဆပါ) အတွက် Temporal features ထည့်သွင်းခြင်း။
    """
    
    # DataFrame ၏ Index ကို DatetimeIndex အဖြစ် စစ်ဆေးခြင်း
    if not isinstance(df_5m.index, pd.DatetimeIndex):
         raise TypeError("DataFrame ၏ Index သည် DatetimeIndex ဖြစ်ရပါမည်။")

    df_5m.index = df_5m.index.tz_localize(None)
    # Index ကို Timezone aware (UTC) အဖြစ် သေချာအောင်လုပ်ခြင်း
    if df_5m.index.tz is None:
        # Timezone-Naive data ကို မူရင်း Source Timezone ဖြင့် localize
        # Dukascopy data ဆိုရင် 'UTC' သုံးပြီး၊ Broker data ဆိုရင် 'GMT+3' လိုမျိုး သုံးနိုင်ပါတယ်
        df = df_5m.tz_localize(source_tz, ambiguous='NaT', nonexistent='NaT')
        df = df.tz_convert('UTC')
    else:
        df = df_5m.copy()
    
    # ----------------------------------------------------
    # I. အခြေခံ features နှင့် Cyclical Encoding များ (Hour ကို Index မှ တိုက်ရိုက်ယူခြင်း)
    # ----------------------------------------------------

    df['weekday'] = df.index.dayofweek 
    df['day'] = df.index.day
    df['week'] = df.index.isocalendar().week.astype(int)
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['hour'] = df.index.hour
    
    # နာရီအတွက် Cyclical features
    df['hour_sin'] = np.sin(2 * np.pi * df['hour']/24).round(6)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour']/24).round(6)

    # ----------------------------------------------------
    # III. DST-Aware Market Sessions (Timezone Handling)
    # ----------------------------------------------------
    
    # နာရီကို local time zone သို့ ပြောင်းလဲ (Timezone Aware Index မှသာ tz_convert လုပ်နိုင်သည်)
    london_time = df.index.tz_convert('Europe/London')
    ny_time = df.index.tz_convert('America/New_York')

    # Session Hours (cf မှ Local Time နာရီများကို ပေးပို့ရပါမည်။)
    ny = cf.trading_hour('NewYork')
    ldn = cf.trading_hour('London')

    # London Session (Local Time: 08:00 - 16:00)
    df['london_session'] = ((london_time.hour >= ldn['from']) & (london_time.hour < ldn['to'])).astype(int)
    
    # NY Session (Local Time: 13:00 - 21:00 UTC/GMT) -> (9:00 - 17:00 EST/EDT)
    # cf မှ Local NY Time (ဥပမာ: 9, 17) ကို ပေးပို့ရမည်
    df['ny_session'] = ((ny_time.hour >= ny['from']) & (ny_time.hour < ny['to'])).astype(int)

    df['overlap_session'] = (df['london_session'] & df['ny_session']).astype(int)

    # ... (IV. Holiday features ကို ဆက်လက်ထည့်သွင်းနိုင်သည်) ...
    
    #df['symbol'] = symbol
    
    # ယာယီ columns များကို ဖယ်ရှားခြင်း
    df = df.drop(columns=['minute_block_15'], errors='ignore') # minute_block_15 သည် 1M data မှ လာလျှင်သာ လိုအပ်သည်။ 5M တွင် မလိုအပ်ပါ။
    
    # Index ကို reset မလုပ်ဘဲ ပြန်ပို့ပါ (Env အတွက် Datetime Index လိုအပ်သည်)
    return df

In [110]:
from finta import TA

def tech_indicators(df, cf=None):
    """
    Forex RL အတွက် Price Action၊ Momentum နှင့် Long-Term Trend Features များကို ထည့်သွင်းခြင်း။
    """
    
    sma_fast_period = cf.indicator('sma_fast_period')
    sma_mid_period = cf.indicator('sma_mid_period')
    sma_slow_period = cf.indicator('sma_slow_period')
    atr_period = cf.indicator('atr_period')
    rsi_period = cf.indicator('rsi_period')
    macd_fast_period = cf.indicator('macd_fast_period')
    macd_slow_period = cf.indicator('macd_slow_period')
    macd_signal_period = cf.indicator('macd_signal_period')
    adx_period = cf.indicator('adx_period')
    stoch_period = cf.indicator('stoch_period')

    
    # --- ၁။ Volatility Measure (ATR ကို Base အဖြစ် အသုံးပြုရန်) ---
    df['atr_base'] = TA.ATR(df, period=atr_period).ffill()

    # --- ၂။ Price Action Features ---
    df['log_returns'] = np.log(df['close'] / df['close'].shift(1)).ffill().round(6)
    df['price_norm'] = (df['close'] - df['close'].shift(sma_fast_period)) / df['atr_base'] # 100-bar SMA ကို ခန့်မှန်းခြေသုံး
    df['spread_ratio'] = (df['high'] - df['low']) / df['atr_base']
    df['body_ratio'] = (df['close'] - df['open']) / (df['high'] - df['low']).replace(0, 1e-6)

    # --- ၃။ Momentum & Trend Features (SMA 100/200 Cross ကို ထည့်သွင်းခြင်း) ---

    df['rsi'] = TA.RSI(df, period=rsi_period).ffill().round(6)
    macd_data = TA.MACD(df, period_fast=macd_fast_period, period_slow=macd_slow_period, signal=macd_signal_period)
    df['macd_hist'] = macd_data.SIGNAL.ffill().round(6)
    
    # Trend ရှိခြင်း/မရှိခြင်း နှင့် Trend ၏ ခိုင်မာမှု ကို တိုင်းတာခြင်း။
    # SMA Cross က Trend Direction ကို ပေးသော်လည်း၊ 
    # ADX က Direction ရဲ့ Strength ကို ပေးသည်။ 
    # ADX နိမ့်ပါက Range/Sideways ဖြစ်ပြီး၊ 
    # ADX မြင့်ပါက Strong Trend ဖြစ်ကြောင်း Agent ကို သိစေသည်။
    df['adx'] = TA.ADX(df, period=adx_period)  
    
    # RSI ဖြင့် Overbought/Oversold ကို စစ်ဆေးပြီး၊ 
    # Stochastic ဖြင့် Momentum Change ကို အတည်ပြုနိုင်သည်။ 
    # နှစ်ခုစလုံး Extreme Level တွင် ရှိနေပါက Reversal ဖြစ်နိုင်ခြေ ပိုများကြောင်း Agent က သင်ယူနိုင်သည်။
    df['stoch_k'] = TA.STOCH(df, period=stoch_period)   
    # *** သင်တောင်းဆိုသော Long-Term SMA Cross Feature အသစ် ***

    df['ma_fast'] = TA.SMA(df, period=sma_mid_period)
    df['ma_slow'] = TA.SMA(df, period=sma_slow_period)
    
    # SMA 100 နှင့် SMA 200 တို့၏ ကွာခြားချက်ကို ATR ဖြင့် Normalization လုပ်ခြင်း
    df['ma_cross'] = ((df['ma_fast'] - df['ma_slow']) / df['atr_base']).ffill().round(6)
    
    # --- ၄။ Data Cleaning ---
    df = df.replace([np.inf, -np.inf], np.nan).fillna(0) 

    numeric_cols = df.select_dtypes(include=[np.number]).columns
    df[numeric_cols] = df[numeric_cols].clip(lower=-1e5, upper=1e5).round(6)
    
    return df

In [115]:
import pandas as pd
import numpy as np

def add_news_feature(df_price: pd.DataFrame, df_news: pd.DataFrame, ahead_bars: int = 6, behind_bars: int = 6) -> pd.DataFrame:
    """
    High-Impact News Event မတိုင်မီနှင့် အပြီးတွင် Binary Feature (1) ကို ထည့်သွင်းခြင်း။ 
    searchsorted ကိုသုံး၍ Timezone Conflict များကို ရှောင်ရှားခြင်း။
    """
    
    # 1. News Feature Column ကို 0 ဖြင့် စတင်ဖြည့်သွင်းခြင်း
    if 'is_high_impact_news' not in df_price.columns:
        df_price['is_high_impact_news'] = 0

    # 2. News 'Start' Column ကို Datetime သို့ ပြောင်းလဲခြင်း
    try:
        df_news['Start_dt'] = pd.to_datetime(df_news['Start'])
    except Exception as e:
        print(f"Error converting news 'Start' column to datetime: {e}")
        return df_price

    # 3. Timezone ကို ကိုင်တွယ်ခြင်း (Price Index Timezone အတိုင်း ညှိခြင်း)
    price_index = df_price.index
    
    # Price Index မှာ Timezone ရှိမရှိ စစ်ဆေးခြင်း
    if price_index.tz is not None:
        # News Time ကို Price Index ရဲ့ Timezone အတိုင်း ပြောင်းလဲပေးခြင်း
        df_news['Start_dt'] = df_news['Start_dt'].dt.tz_localize('UTC').dt.tz_convert(price_index.tz) 
    else:
        # Price Index မှာ Timezone မရှိရင် News Time ကနေ Timezone ကို ဖယ်ထုတ်ခြင်း
        df_news['Start_dt'] = df_news['Start_dt'].dt.tz_localize(None)

    # 4. Binary Search (searchsorted) ဖြင့် Location ရှာပြီး Feature ထည့်သွင်းခြင်း
    
    for news_time in df_news['Start_dt']:
        
        # searchsorted သည် Integer Index Location ကို ပြန်ပေးသောကြောင့် get_loc() ၏ method argument error ကို ရှောင်ရှားနိုင်သည်
        loc = price_index.searchsorted(news_time, side='left') 
        
        # loc သည် Index ရဲ့ အပြင်ဘက်ကို ရောက်သွားနိုင်သောကြောင့် အနားသတ် စစ်ဆေးခြင်း
        if loc >= len(price_index):
            continue

        # News Event မတိုင်မီ bars
        start_index = max(0, loc - ahead_bars)
        
        # News Event အပြီး bars
        end_index = min(len(price_index), loc + behind_bars)
        
        # 1 Value ကို သတ်မှတ်ထားသော Range အတွင်း ထည့်သွင်းခြင်း (Integer Location ဖြင့်)
        df_price.iloc[start_index:end_index, df_price.columns.get_loc('is_high_impact_news')] = 1

    return df_price

In [111]:
file_path_from_data = "./drive/MyDrive/data/raw/EURUSD_2020_all.csv"
file_path_to_data = "./drive/MyDrive/data/raw/EURUSD_2020_all_5.csv"

df_all_5m_data = convert_1m_csv_to_5m_df(file_path_from_data)
df_all_5m_data.to_csv(file_path_to_data)


✅ Conversion successful! 5-Min rows: 149715


In [112]:
cf = EnvConfig('./drive/MyDrive/configure.json')

raw = pd.read_csv(file_path_to_data)

df = patch_missing_data(raw,cf=cf)

shape of  column: 6
Week 2020-01-05 00:00:00+00:00 has 576/1440 bars
Week 2020-01-12 00:00:00+00:00 has 1439/1440 bars
Week 2020-03-29 00:00:00+00:00 has 1433/1440 bars
Week 2020-09-20 00:00:00+00:00 has 1439/1440 bars
Week 2020-10-04 00:00:00+00:00 has 1436/1440 bars
Week 2020-12-27 00:00:00+00:00 has 1152/1440 bars
Week 2021-01-03 00:00:00+00:00 has 1142/1440 bars
Week 2021-05-23 00:00:00+00:00 has 1439/1440 bars
Week 2021-05-30 00:00:00+00:00 has 1437/1440 bars
Week 2021-06-06 00:00:00+00:00 has 1439/1440 bars
Week 2021-09-19 00:00:00+00:00 has 1439/1440 bars
Week 2021-10-10 00:00:00+00:00 has 1439/1440 bars
Week 2021-12-12 00:00:00+00:00 has 1439/1440 bars
Week 2021-12-26 00:00:00+00:00 has 1438/1440 bars
Week 2022-01-02 00:00:00+00:00 has 1428/1440 bars


In [113]:
# Broker Data (00:00 မှ စသော) ကို ခေါ်ဆိုသည့်အခါ
# GMT+2/GMT+3 ကို အလိုအလျောက် ကိုင်တွယ်ပေးရန်
axiory_tz = 'Europe/Kiev'  

dft = add_time_feature(df, cf=cf, source_tz=axiory_tz)

In [114]:
df = tech_indicators(dft, cf=cf)

In [116]:
file_path_from_news = "./drive/MyDrive/data/raw/2020_2021_News.csv"
news = pd.read_csv(file_path_from_news)
df_news = add_news_feature(df, news)
df_news.to_csv('final.csv')

In [ ]:


cf = EnvConfig('./drive/MyDrive/configure.json')

raw = pd.read_csv(file_path_to_data)

df = patch_missing_data(raw,cf=cf)

symbol = 'EURUSD'
# Broker Data (00:00 မှ စသော) ကို ခေါ်ဆိုသည့်အခါ
# GMT+2/GMT+3 ကို အလိုအလျောက် ကိုင်တွယ်ပေးရန်
axiory_tz = 'Europe/Kiev'  

dft = add_time_feature(df, cf=cf, source_tz=axiory_tz)
dft.head(3)



cf = EnvConfig('./drive/MyDrive/configure.json')

df = tech_indicators(dft, cf=cf)
df.to_csv('tech.csv')

,Id,Start,Name,Impact,Currency
0,be36572c-0dbf-4b11-aca1-bf854160ba4a,01/03/2020 13:00:00,Harmonized Index of Consumer Prices (YoY),HIGH,EUR
1,5b125588-01ca-4aca-a365-8902ef7839f1,01/03/2020 15:00:00,ISM Manufacturing PMI,HIGH,USD
2,3fa72395-467b-48b9-b0a3-29529d398237,01/03/2020 19:00:00,FOMC Minutes,HIGH,USD
3,2d6440df-69b8-4322-b175-002ae17088e3,01/07/2020 10:00:00,Core Harmonized Index of Consumer Prices (YoY),HIGH,EUR


In [91]:
df.head(1)

,open,high,low,close,vol,weekday,day,week,month,year,...,atr_base,log_returns,price_norm,spread_ratio,body_ratio,rsi,macd_hist,adx,stoch_k,ma_cross
2020-01-01 22:00:00+00:00,1.12117,1.12128,1.12087,1.12114,31.0,2,1,1,1,2020,...,0.0,0.0,0.0,0.0,-0.073171,0.0,0.0,0.0,0.0,0.0


In [92]:
a = df.index.get_loc('01/03/2020 13:00:00')
a # 2020-01-03 12:50:00+00:00,

468

In [ ]:
df_news = add_news_feature(df, news)
df_news.to_csv('df_new.csv')

In [103]:
df_news.head(4)
df_news.to_csv('df_new.csv')

In [97]:
import pandas as pd
print(pd.__version__)

2.2.3


In [90]:
for news_time in news['Start']:
    print(type(news_time))
    print(news_time)

<class 'str'>
01/03/2020 13:00:00
<class 'str'>
01/03/2020 15:00:00
<class 'str'>
01/03/2020 19:00:00
<class 'str'>
01/07/2020 10:00:00
<class 'str'>
01/07/2020 10:00:00
<class 'str'>
01/07/2020 15:00:00
<class 'str'>
01/10/2020 13:30:00
<class 'str'>
01/10/2020 13:30:00
<class 'str'>
01/14/2020 13:30:00
<class 'str'>
01/14/2020 13:30:00
<class 'str'>
01/16/2020 07:00:00
<class 'str'>
01/16/2020 12:30:00
<class 'str'>
01/16/2020 18:00:00
<class 'str'>
01/20/2020 18:30:00
<class 'str'>
01/21/2020 09:00:00
<class 'str'>
01/21/2020 10:00:00
<class 'str'>
01/23/2020 12:45:00
<class 'str'>
01/23/2020 12:45:00
<class 'str'>
01/23/2020 13:30:00
<class 'str'>
01/24/2020 08:30:00
<class 'str'>
01/24/2020 08:30:00
<class 'str'>
01/24/2020 09:00:00
<class 'str'>
01/24/2020 10:30:00
<class 'str'>
01/28/2020 13:30:00
<class 'str'>
01/29/2020 19:00:00
<class 'str'>
01/29/2020 19:30:00
<class 'str'>
01/30/2020 13:00:00
<class 'str'>
01/30/2020 13:30:00
<class 'str'>
01/31/2020 10:00:00
<class 'str'>


In [104]:
def render_to_file(**kwargs):
    log_header                  =   kwargs.get("log_header",False)
    log_filename                =   kwargs.get("log_filename","")
    printout                    =   kwargs.get("printout",False)
    balance                     =   kwargs.get("balance")
    balance_initial             =   kwargs.get("balance_initial")
    transaction_close_this_step =   kwargs.get("transaction_close_this_step",[])
    done_information            =   kwargs.get("done_information","")
    profit                      =   balance - balance_initial

    tr_lines                    =   ""
    tr_lines_comma              =   ""
    _header                     =   ""
    _header_comma               =   ""
    if log_header:
        _header = f'{"Ticket":>8} {"Type":>4} {"ActionStep":16} \
                    {"ActionPrice":>12} {"CloseStep":8} {"ClosePrice":>12} \
                    {"OpenBal":>12} {"CloseBal":>12} {"Status":8} {"Info":>8} {"PIPS":>6} {"SL":>6} {"PT":>6} {"DeltaStep":8}\n'


        _header_comma = f'{"Ticket,Type,ActionTime,ActionStep,ActionPrice,CloseTime,ClosePrice, OpenBal, CloseBal, Status, Info, PIPS,SL,PT,CloseStep,DeltaStep"}\n'
    if transaction_close_this_step:
        for _tr in transaction_close_this_step:
            if _tr["CloseStep"] >=0:
                tr_lines += f'{_tr["Ticket"]:>8} {_tr["Type"]:>4} {_tr["ActionStep"]:16} \
                    {_tr["ActionPrice"]:.5f} {_tr["CloseStep"]:8} {_tr["ClosePrice"]:.5f} \
                    {_tr["OpenBal"]:.2f} {_tr["CloseBal"]:.2f} {_tr["Status"]:8}  {_tr["Info"]:>8}  {_tr["PIPS"]:4.0f} {_tr["SL"]:4.0f} {_tr["PT"]:4.0f} {_tr["DeltaStep"]:8}\n'

                tr_lines_comma += f'{_tr["Ticket"]},{_tr["Type"]},{_tr["ActionTime"]},{_tr["ActionStep"]}, \
                    {_tr["ActionPrice"]},{_tr["CloseTime"]},{_tr["ClosePrice"]}, \
                    {_tr["OpenBal"]},{_tr["CloseBal"]}, {_tr["Status"]},{_tr["Info"]},{_tr["PIPS"]},{_tr["SL"]},{_tr["PT"]},{_tr["CloseStep"]},{_tr["DeltaStep"]}\n'

    log = _header_comma + tr_lines_comma
    # log = f"Step: {current_step}   Balance: {balance}, Profit: {profit} \
    #     MDD: {max_draw_down_pct}\n{tr_lines_comma}\n"
    if done_information:
        log += done_information
    if log:
        # os.makedirs(log_filename, exist_ok=True)
        dir_path = os.path.dirname(log_filename)
        if dir_path and not os.path.exists(dir_path):
            os.makedirs(dir_path, exist_ok=True)
        with open(log_filename, 'a+') as _f:
            _f.write(log)
            _f.close()

    tr_lines = _header + tr_lines
    if printout and tr_lines:
        print(tr_lines)
        if done_information:
            print(done_information)

In [ ]:
import os
import json

import logging
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
# from stable_baselines3.common.callbacks import LearningRateSchedule
from stable_baselines3 import PPO
import gymnasium as gym
from gymnasium import spaces
import torch.nn.functional as F
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import datetime

class ForexTradingEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, file, cf, asset, logger_show=False, save_plot=False, scaler=None):
        # scaler parameter ကို ထပ်ထည့်ရပါမည်။ ၎င်းသည် Global Train Set တွင် Fit လုပ်ထားသော Scaler ဖြစ်ရမည်။
        # 'scaler' သည် Global Train Set တွင် Fit လုပ်ထားသော StandardScaler instance ဖြစ်ရမည်။
        self.scaler = scaler
        if self.scaler is None:
             raise ValueError("A fitted StandardScaler instance must be provided to the Environment.")  
         
        super(ForexTradingEnv, self).__init__()
        # ကိန်းရှင်များကို စတင်သတ်မှတ်သည်။
        self._initialize_parameters(file, cf, asset, logger_show, save_plot)
        
        # [NEW ACTION] Raw Data ကို Scaler ဖြင့် Transform လုပ်ခြင်း
        # OHLCV features များကို Scaling လုပ်ရပါမယ် (ဥပမာ: open, high, low, close, vol, atr_base, log_returns, price_norm, etc.)
        # Scaling လုပ်ရန် features များစာရင်းကို သတ်မှတ်ပါ။
        # ဤနေရာတွင် ကျွန်တော်တို့သည် OHLCV နှင့် Indicator Features အားလုံးကို Scaling လုပ်မည်ဟု ယူဆပါမည်။
        self._scale_data()
        
        
        # Action နှင့် Observation Spaces ကို သတ်မှတ်သည်။
        self._initialize_spaces()
        # Environment ကို အစပြုအခြေအနေသို့ ပြန်လည်သတ်မှတ်သည်။
        self.reset()


    def _scale_data(self):
        """Raw Data (self.data) မှ features များကို Global Scaler ဖြင့် Transform လုပ်ခြင်း"""
        # [self.features] တွင် OHLCV နှင့် Indicator Features အားလုံး ပါဝင်ရမည်။
        # [NOTE]: 'time' ကဲ့သို့သော Non-Numeric features များကို self.features တွင် မပါဝင်ကြောင်း သေချာပါစေ။
        if not self.scaler.scale_.any():
             logger.warning("Scaler is not properly fitted. Continuing with raw data.")
             return
             
        # Scaled Features များကို Original DataFrame တွင် အစားထိုးသည်။
        # Scaled Data တွင် NaN/Inf မဖြစ်စေရန် Data Frame ကို ကြိုတင်စစ်ဆေးထားသင့်ပါသည်။
        self.data[self.features_scaled] = self.scaler.transform(self.data[self.features_scaled])
        # logger.info(f"Data scaled successfully using fitted StandardScaler.")
        
    # ကိန်းရှင်များကို စတင်သတ်မှတ်သည်။
    def _initialize_parameters(self, file, cf, asset, logger_show, save_plot):
        # Params to variables
        self.csv_file               =   file
        self.cf                     =   cf
        self.symbol_col             =   asset
        self.features_scaled        =   self.cf.env_parameters('features_scaled') # Time-Series Features List
        self.features_unscaled      =   self.cf.env_parameters('features_unscaled')
        self.features_filter        =   self.cf.env_parameters('features_filter')
        # Scaled Data Frame ၏ Feature List
        self.obs_features           =   self.features_scaled + self.features_unscaled
        self.sequence_length        =   self.cf.data_processing_parameters("sequence_length") # Transformer Lookback Window (100)
        self.logger_show            =   logger_show
        self.save_plot              =   save_plot

        self.data_raw = pd.read_csv(file)
        if 'time' in self.data_raw.columns:
            self.data_raw = self.data_raw.set_index(pd.to_datetime(self.data_raw['time'], utc=True)).drop(columns=['time'])
        
        # self.data ကို Scaling လုပ်ရန် Copy ယူပါမည်။
        self.data = self.data_raw.copy()
            
        # We use sequence transformer, so max steps will be this
        self.max_steps              =   len(self.data) - self.sequence_length - 1

        # Configs to variables
        # Agent က Action က Continuous Action ကို Discrete Action သို့ပြောင်းပေးသော threshold
        self.action_threshold       =   self.cf.env_parameters('action_threshold')
        self.balance_initial        =   self.cf.env_parameters('balance')

        # position close မဖြစ်သေးရင်
        # buy ထားပြီး price up ဖြစ်နေရင် reward ပေး။ sell ထားပြီး price down ဖြစ်နေရင် reward ပေး
        # position management မှာလည်း သုံး။
        # buy မှာ မြတ်နေရင် tp အပေါ်ရွေ့ sl အပေါ်ရွေ့။  ရှုံးနေရင် tp အောက်ရွေ့ sl အပေါ်တင်,
        # sell မှာ မြတ်နေရင် tp အောက်ရွေ့ sl အောက်ရွေ့။ ရှုံးနေရင် tp အပေါ်တင် sl အောက်ချ
        self.good_position_reward_scale = self.cf.env_parameters("good_position_reward_scale") # ဥပမာ: 0.01
        # ရည်ရွယ်ချက် ၂: SL/PT Trailing အတွက် တန်ဖိုး (Move Step Size)
        self.trailing_distance = self.cf.env_parameters("trailing_stop_distance_points")

        # အရှုံးနဲ့အမြတ် မျှတမှုရှိတဲ့ trading performance အတွက် ပေးတဲ့ bonus reward 0.01
        # self.consistency_reward = self.cf.env_parameters("consistency_reward")
        self.stop_loss = self.cf.symbol(self.symbol_col, "stop_loss_max")
        self.profit_taken = self.cf.symbol(self.symbol_col, "profit_taken_max")
        self.point = self.cf.symbol(self.symbol_col, "point")
        self.transaction_fee = self.cf.symbol(self.symbol_col, "transaction_fee")
        self.over_night_penalty = self.cf.symbol(self.symbol_col, "over_night_penalty")
        self.max_current_holding = self.cf.symbol(self.symbol_col, "max_current_holding")
        # Drawdown Penalty Factor
        self.drawdown_penalty_factor = self.cf.env_parameters("drawdown_penalty_factor")
        self.margin_requirement = self.cf.env_parameters('margin_requirement')


    # Action နှင့် Observation Spaces ကို သတ်မှတ်သည်။
    def _initialize_spaces(self):
        # Continuous actions: [1 -> 0.5] LONG | [0.5 -> -0.5] HOLD |[-0.5 -> -1] SHORT
        self.action_space = spaces.Box(
            low=-1,
            high=1,
            shape=(1,),
            dtype=np.float32
        )

        # [MODIFIED] Transformer Observation Space: Time Series (100) + Context (4)
        N_FEATURES_TS = len(self.obs_features)
        N_FEATURES_CONTEXT = 4 # [Equity, Drawdown, Open_Pos_Ratio, Time_Context]
        
        # [MODIFIED] Observation Space (Time Series Sequence Only)
        # Transformer သုံးထားသော features တွေရဲ့ previous sequence length candle ကိုပါ တပြိုင်တည်းကြည့်
        obs_shape = (self.sequence_length, N_FEATURES_TS + N_FEATURES_CONTEXT)
        
        self.observation_space = spaces.Box(
            low=-np.inf, 
            high=np.inf, # Scaled Data များသည် Theoretical Inf/ -Inf ရှိနိုင်သောကြောင့် np.inf ကို သုံးပါ
            shape=obs_shape,
            dtype=np.float32
        )

    # Environment ကို အစပြုအခြေအနေသို့ ပြန်လည်သတ်မှတ်သည်။
    def reset(self, *, seed = None, options = None):
        super().reset(seed=seed, options=options)

        self.ticket_id          =   0
        self.ttl_rewards        =   0 # total rewards

        self.balance            =   self.balance_initial
        self.positions          =   []

        # equity tracking
        self.equity_curve       =   [self.balance_initial] # Starting with initial balance
        # အမြင့်ဆုံးရောက်ဖူးတဲ့ eq value
        self.peak_equity        =   self.balance_initial # Start with initial balance as peak

        self.max_drawdown       =   0.0
        self.current_drawdown   =   0.0

        # transformer အသုံးပြုထားခြင်းကြောင့်
        self.current_step       =   self.sequence_length
        logger.info(f"--- Environment reset. Starting at step {self.current_step} --total rewards: {self.ttl_rewards}")

        observation             =   self._next_observation()
        info                    =   {}
        return  observation, info


# AI model အတွက် လက်ရှိ market condition ကိုကိုယ်စားပြုတဲ့ observation data ကို ပြင်ဆင်ပေးဖို့ဖြစ်ပါတယ်။
    def _next_observation(self):
        
        # 1. Time Series Observation (Scaled Data)
        obs_ts = self.data.iloc[
            self.current_step - self.sequence_length: self.current_step
        ][self.obs_features].values # Shape: (100, N_Features_TS)

        # 2. Account State (Non-Time-Series / Context Vector)
        current_equity = self._calculate_current_equity()
        open_positions_count = sum(1 for p in self.positions if p['Status'] == 0)

        obs_context = np.array([
            current_equity / self.balance_initial, # 1. Normalized Equity
            self.current_drawdown,                 # 2. Current Drawdown (Percentage)
            open_positions_count / self.max_current_holding, # 3. Open Positions Ratio
            self.data.iloc[self.current_step]['hour_cos']   # 4. Time Context (Scaled)
        ], dtype=np.float32) # Shape: (4,)

        # 3. Final Observation Construction (Time Series + Context)
        
        # Context features များကို Sequence Length (100) အလိုက် ဖြန့်ကျက်ခြင်း (Broadcasting)
        obs_context_expanded = np.tile(obs_context, (self.sequence_length, 1)) # Shape: (100, 4)
        
        # Horizontal Stack (Sequence length, N_Features_TS + N_Features_Context)
        obs_final = np.hstack([obs_ts, obs_context_expanded])

        # 4. PyTorch/Device Conversion and Validation
        try:
            # NumPy array → PyTorch tensor ပြောင်းမယ်
            obs_tensor = torch.tensor(obs_final, dtype=torch.float32).to(device)
            # Data Validation စစ်ဆေးခြင်း
            if torch.isnan(obs_tensor).any() or torch.isinf(obs_tensor).any():
                logger.error(f"Invalid observation (NaN/Inf) at step {self.current_step}")
                raise ValueError(f"Invalid observation (NaN/Inf) at step {self.current_step}")
            return obs_tensor.cpu().numpy()
            
        except NameError:
            # Torch ကို မသုံးပါက NumPy ကိုသာ ပြန်ပေးပါ။
            if np.isnan(obs_final).any() or np.isinf(obs_final).any():
                logger.error(f"Invalid observation (NaN/Inf) at step {self.current_step}")
                raise ValueError(f"Invalid observation (NaN/Inf) at step {self.current_step}")
            return obs_final # Final NumPy array
        
            
    def _ray_mask(self, a, c, bounds):
        """
        Ray Mask လုပ်ပါ။
        a: မူရင်း action (np.array)
        c: သက်ဆိုင်ရာ အစု ရဲ့ အလယ်ဗဟို
        A_r_boundary_func: lambda_A_r တွက်တဲ့ func
        A_boundary_func: lambda_A တွက်တဲ့ func
        """
        if np.allclose(a, c):
            return c  # အလယ်မှာ ဆို မရွှေ့

        direction = a - c
        norm_dir = direction / np.linalg.norm(direction)

        lambda_A_r = bounds[1] - c if norm_dir > 0 else c - bounds[0]
        lambda_A = 1 - c if norm_dir > 0 else c - (-1)


        if lambda_A_r <= 0 or lambda_A <= 0:
            return c  # အမှား ရှောင်

        scale = lambda_A_r / lambda_A
        a_r = c + scale * direction
        return np.clip(a_r, -1, 1)  # Action space ကန့်သတ်


    def _get_action_name(self, _action, ma_first, ma_slow):

        c = 0.0  # အလယ်ဗဟို (hold)
        if ma_first > ma_slow:  # Uptrend: buy only [0, 1]
            bounds = [0, 1]
        else:  # Downtrend: sell only [-1, 0]
            bounds = [-1, 0]

        a_masked = self._ray_mask(_action, c, bounds)

        """Convert continuous action to discrete action name"""
        if a_masked >= self.action_threshold:
            return "BUY"
        elif a_masked <= -self.action_threshold:
            return "SELL"
        else:
            return "HOLD"

    def step(self, action):
        # self.data သည် Index တွင် 'time' ကို ထားရှိပြီး drop လုပ်ထားသောကြောင့်၊ Index မှ time ကို ယူရန်လိုသည်။
        current_row_raw = self.data_raw.iloc[self.current_step]
        
        # Unscaled Price Features
        _o, _h, _l, _c, ma_fast, ma_slow = current_row_raw[['open', 'high', 'low', 'close', 'ma_fast', 'ma_slow']]
        
        _t = self.data.index[self.current_step] # Get time from index    
            
        reward                      =   0 # ဒီ step အတွက် စုစုပေါင်း reward
        position_reward             =   0 # Position ပိတ်ရင် ရတဲ့ reward
        action_hold_reward          =   0 # Hold action အတွက် reward/penalty

        _msg                        =   []
        _action                     =   action[0] # action value eg. [0.75]
        open_position               =   0
        for position in self.positions:
            if position['Status']   ==  0:
                position_reward, closed, _msg   =   self._calculate_reward(position)
                if not closed: open_position += 1  # Count what we already knew
                reward += position_reward

        # Continuous actions: [1 -> 0.5] LONG | [0.5 -> -0.5] HOLD |[-0.5 -> -1] SHORT
        action_name = self._get_action_name(action, ma_fast, ma_slow)

        if open_position < self.max_current_holding and action_name in ['BUY', 'SELL']:
            self.ticket_id  +=  1

            # Real trading မှာ margin requirement ရှိသလိုမျိုး
            # Position ဖွင့်ရင် capital ချုပ်ငြားနေရတယ်
            # Position ပိတ်တဲ့အခါ ပြန်ပေါင်းထည့်ပေးတယ်
            self.balance -= self.margin_requirement # hold up, this will make sure model can not open a lot of

            position        =   {
                "Ticket"        :   self.ticket_id,
                "Symbol"        :   self.symbol_col,
                "ActionTime"    :   _t,
                "Type"          :   action_name,
                "Lot"           :   1,
                "ActionPrice"   :   _c,
                "SL"            :   self.stop_loss,
                "PT"            :   self.profit_taken,
                "MaxDD"         :   0,
                "Swap"          :   0.0,
                "CloseTime"     :   "",
                "ClosePrice"    :   0.0,
                "Point"         :   self.point,
                "Reward"        :   self.transaction_fee,
                "DateDuration"  :   _t.date().isoformat(),
                "Status"        :   0, # 0 is Position is currently OPEN and active
                #"PIPS"          :   self.transaction_fee, # Price Interest Point (profit/loss ကို measure လုပ်တဲ့ unit)
                "PIPS"          :   0,
                "ActionStep"    :   self.current_step,
                "CloseStep"     :   -1, # Step number when position closed, not close yet is -1
                "DeltaStep"     :   0,
                "OpenBal"       :   self.balance,
                "CloseBal"       :   0,
                "HighestPrice"  :   _c,
                "LowestPrice"   :   _c,
            }

            self.positions.append(position)
            # do not use transaction_fee penalty
            # reward = self.transaction_fee #open cost
            # model က အလွန်အကျွံ position တွေ မဖွင့်မိအောင် ထိန်းချုပ်တဲ့ mechanism ဖြစ်ပါတယ်။
            _msg.append(f'Step:{self.current_step} Tkt:{position["Ticket"]} {position["Type"]} Rwd:{position["PIPS"]} SL:{position["SL"]} PT:{position["PT"]}')

        # HOLD Penalty ကို အလွန်သေးငယ်သော တန်ဖိုး
        # (ဥပမာ: -0.0001) သို့ ပြောင်းပါ။ အကောင်းဆုံးမှာ
        # Trading မလုပ်ခြင်းအတွက် Penalty မပေးဘဲ action_hold_reward = 0 ထားပါ။
        elif open_position < self.max_current_holding and action_name == "HOLD":
            action_hold_reward  =   0  # no open any position, encourage open position
        else:
            action_hold_reward  =   0




        reward              +=  action_hold_reward

        # Move to the next time step
        self.current_step   +=  1

        # check if episode is done
        terminated          =   (self.balance <= 0)
        truncated           =   (self.current_step > self.max_steps)

        # get next observation
        obs                 =   self._next_observation()
        _msg.append(f'---idle----step:{self.current_step}, RF:{action_name} Action:{_action} Balance: {self.balance} reward:{reward} total_rewards:{self.ttl_rewards} position_reward:{position_reward} action_hold_reward:{action_hold_reward}')


        current_equity = self._calculate_current_equity()
        self.equity_curve.append(current_equity)
        self._calculate_drawdown()  # This updates peak_equity and drawdowns

        # =========================================================================
        # START: Drawdown Penalty Logic
        # =========================================================================
        # self.current_drawdown သည် Percentage (0.0 မှ 1.0) ဖြစ်သည်။


        drawdown_penalty = self.current_drawdown * self.drawdown_penalty_factor
        # Reward တွင် နုတ်ပေးခြင်း
        reward -= drawdown_penalty

        # Log the penalty for debugging
        _msg.append(f'Drawdown Penalty: -{drawdown_penalty:.4f} (DD:{self.current_drawdown:.4f})')
        # =========================================================================
        # END: Drawdown Penalty Logic
        # =========================================================================
        # Drawdown Penalty နုတ်ပြီးမှသာ စုစုပေါင်း Reward ကို အပ်ဒိတ်လုပ်ပါ
        self.ttl_rewards += reward  # <--- ဤနေရာတွင် ပြန်ထည့်ပါ

        info = {}

        if terminated or truncated:
            buy_positions = [p for p in self.positions if p["Type"] == "BUY"]
            sell_positions = [p for p in self.positions if p["Type"] == "SELL"]

            buy_count = len(buy_positions)
            sell_count = len(sell_positions)
            total_positions = len(self.positions)

            # Calculate win rates
            buy_wins = len([p for p in buy_positions if p["PIPS"] > 0])
            sell_wins = len([p for p in sell_positions if p["PIPS"] > 0])

            buy_win_rate = buy_wins / buy_count if buy_count > 0 else 0
            sell_win_rate = sell_wins / sell_count if sell_count > 0 else 0

            _m = f'--- Positions: {total_positions} (Buy:{buy_count}, Sell:{sell_count}) | '
            _m += f'WinRates: Buy:{buy_win_rate:.1%}, Sell:{sell_win_rate:.1%} | '
            _m += f'TotalRewards: {self.ttl_rewards} Balance: {self.balance}'

            logger.info(_m)
            _msg.append(_m)

            # Additional info
            if self.logger_show:
                for _m in _msg:
                    logger.info(_m)

            info["info"]                = _msg
            info["sharpe"]              = self._calculate_sharpe()  # ✅ Now works! 💡 'sharpe_ratio' မှ 'sharpe' သို့ပြောင်းပါ။
            info["drawdown"]            = self.max_drawdown         # ✅ Now accurate!'max_drawdown' မှ 'drawdown' သို့ပြောင်းပါ။
            info["current_equity"]      = current_equity            # ✅ For debugging
            info["peak_equity"]         = self.peak_equity          # ✅ For debugging
            info["equity_curve_length"] = len(self.equity_curve)    # ✅ Monitor growth

        return obs, reward, terminated, truncated, info




    def _calculate_reward(self, position):
        _o, _h, _l, _c              = self.data_raw.iloc[self.current_step][['open', 'high', 'low', 'close']]
        _t                          = self.data.index[self.current_step]
        _msg                        =   []

        entry_price                 =   position['ActionPrice']
        direction                   =   position['Type']
        profit_target_price         =   entry_price + position['PT']/ self.point if direction == 'BUY' else entry_price - position['PT']/self.point
        stop_loss_price             =   entry_price + position['SL']/ self.point if direction == 'BUY' else entry_price - position['SL']/self.point
        closed                      =   False
        close_position_reward       =   0.0
        good_position_reward        =   0.0

        # Check for stoploss hit
        if (direction == 'BUY' and _l <= stop_loss_price) or (direction == 'SELL' and _h >= stop_loss_price):
            close_position_reward   =   position['SL'] # position sl က minus value ဖြစ်တယ်

            position['CloseTime']   =   _t
            position['ClosePrice']  =   stop_loss_price
            position['Status']      =   1   # Status က open ဆို 0 close ဆို 1
            position['CloseStep']   =   self.current_step
            position['PIPS']        =   close_position_reward - self.transaction_fee
            position['DeltaStep']   =   self.current_step - position['ActionStep']
            position['Info']        =   f'{profit_target_price:.5f} | {stop_loss_price:.5f}'

            self.balance            +=  self.margin_requirement + position['PIPS'] # return 100 is margin hold
            position['CloseBal']    =   self.balance
            closed                  =   True
            _msg.append(f'Step:{self.current_step} Tkt:{position["Ticket"]}: Rwd:{position["PIPS"]}, SL:{position["SL"]}, DeltaStep:{position["DeltaStep"]}')

        elif (direction == 'BUY' and _h >= profit_target_price) or (direction == 'SELL' and _l <= profit_target_price):
            close_position_reward   =    position['PT'] # position tp က plus value ဖြစ်တယ်

            position['CloseTime']   =   _t
            position['ClosePrice']  =   profit_target_price
            position['Status']      =   2   # Status က open ဆို 0 close ဆို 1
            position['CloseStep']   =   self.current_step
            position['PIPS']        =   close_position_reward - self.transaction_fee
            position['DeltaStep']   =   self.current_step - position['ActionStep']
            position['Info']        =   f'{profit_target_price:.5f} | {stop_loss_price:.5f}'

            self.balance            +=  self.margin_requirement + position['PIPS'] # return 100 is margin hold
            position['CloseBal']    =   self.balance
            closed                  =   True
            _msg.append(f'Step:{self.current_step} Tkt:{position["Ticket"]}: Rwd:{position["PIPS"]}, SL:{position["SL"]}, DeltaStep:{position["DeltaStep"]}')

        else:
            if self.current_step + 5 + self.sequence_length >= len(self.data):
                close_position_reward   =   (_c - position["ActionPrice"] if direction == 'BUY' else position["ActionPrice"] - _c)* self.point

                position['CloseTime']   =   _t
                position['ClosePrice']  =   _c
                position['Status']      =   3   # Status က open ဆို 0 close ဆို 1, force close 2
                position['CloseStep']   =   self.current_step
                position['PIPS']        =   close_position_reward - self.transaction_fee
                position['DeltaStep']   =   self.current_step - position['ActionStep']
                position['Info']        =   f'{profit_target_price:.5f} | {stop_loss_price:.5f}'
                self.balance            +=  self.margin_requirement + position["PIPS"] # return 100 is margin hold
                position['CloseBal']    =   self.balance

                closed                  =   True
                _msg.append(f'Step:{self.current_step} Tkt:{position["Ticket"]}: Rwd:{position["PIPS"]}, Cls:End, DeltaStep:{position["DeltaStep"]}')

            else:
                # =========================================================================
                # Real Trailing Stop Logic (အမြင့်ဆုံး ရောက်ဖူးသော ဈေးနှုန်းကို မှတ်တမ်းတင်ခြင်း)
                # =========================================================================
                # 1. Highest/Lowest Price Update

                if direction == "BUY":
                  # Buy position အတွက် အမြင့်ဆုံး ရောက်ဖူးသော ဈေးနှုန်းကို မှတ်တမ်းတင်
                  if _c > position["HighestPrice"]:
                      position["HighestPrice"] = _c

                  # 2. New SL Target Price (Trailing Price) ကို တွက်ချက်ခြင်း
                  # New_SL_Price = HighestPrice - (Trailing Distance Pips ကို Price Change သို့ ပြောင်း)
                  trailing_price = position["HighestPrice"] - self.trailing_distance / self.point

                  # 3. SL ကို အဆင့်မြှင့်တင်ခြင်း
                  # လက်ရှိ SL ထက် ပိုကောင်းမှသာ ရွေ့ပါ
                  if trailing_price > stop_loss_price:

                      stop_loss_price = trailing_price
                      # SL_Price အသစ်ကို Points သို့ ပြန်ပြောင်းပြီး position['SL'] ကို အပ်ဒိတ်လုပ်ပါ
                      position["SL"] = (stop_loss_price - entry_price) * self.point
                    #   if position["SL"] > 0:
                    #       position["SL"]    =   -abs(position["SL"])
                      trailing_happened = True
                  else:
                      trailing_happened = False


                elif direction == "SELL":
                  # Sell position အတွက် အနိမ့်ဆုံး ရောက်ဖူးသော ဈေးနှုန်းကို မှတ်တမ်းတင်
                  if _c < position["LowestPrice"]:
                      position["LowestPrice"] = _c

                  # New SL Target Price (Trailing Price) ကို တွက်ချက်ခြင်း
                  trailing_price = position["LowestPrice"] + self.trailing_distance / self.point

                  # SL ကို အဆင့်မြှင့်တင်ခြင်း
                  if trailing_price < stop_loss_price:
                      stop_loss_price = trailing_price
                      # SL_Price အသစ်ကို Points သို့ ပြန်ပြောင်းပြီး position['SL'] ကို အပ်ဒိတ်လုပ်ပါ
                      position["SL"] = (entry_price - stop_loss_price) * self.point
                    #   if position["SL"] > 0:
                    #       position["SL"]    =   -abs(position["SL"])
                      trailing_happened = True
                  else:
                      trailing_happened = False

                # =========================================================================
                # Reward Logic (Trailing လုပ်ခြင်းအတွက် Bonus ပေးခြင်း)
                # =========================================================================
                # Reward Sign ကို ယခင်အတိုင်း တွက်ပါ။
                delta = _c - entry_price
                if direction == "BUY":
                    reward_sign = 1 if delta >= 0 else -1
                elif direction == "SELL":
                    reward_sign = -1 if delta >= 0 else 1

                good_position_reward = reward_sign * self.good_position_reward_scale

                # Trailing အမှန်တကယ် ဖြစ်သွားမှသာ Bonus Reward ကို ပေးပါ
                if trailing_happened:
                    good_position_reward += 0.001

                position['Info']        =   f'{profit_target_price:.5f} | {stop_loss_price:.5f}'
                position['CloseBal']    =   self.balance
                _msg.append(f'Step:{self.current_step} Tkt:{position["Ticket"]}: NO_Close, PT:{position["PT"]}, SL:{position["SL"]}')

        return close_position_reward + good_position_reward, closed, _msg


    def _calculate_sharpe(self, risk_free_rate=0.0):
        """Calculate Sharpe ratio for the current episode"""
        if len(self.equity_curve) < 2:
            return 0.0

        returns = np.diff(self.equity_curve) / self.equity_curve[:-1]

        if np.std(returns) == 0:
            return 0.0

        sharpe = (np.mean(returns) - risk_free_rate) / np.std(returns)
        return float(sharpe * np.sqrt(288))  # Annualized (5-min bars → 288/day)

    def _calculate_drawdown(self):
        """Update max drawdown during episode"""
        current_equity          =   self.equity_curve[-1]
        self.peak_equity        =   max(self.peak_equity, current_equity)
        self.current_drawdown   =   (self.peak_equity - current_equity) / self.peak_equity
        self.max_drawdown       =   max(self.max_drawdown, self.current_drawdown)


    def _calculate_current_equity(self):
        """Calculate total current equity (balance + unrealized P/L)"""
        total_equity = self.balance  # Start with cash balance

        # Add unrealized P/L from open positions
        for position in self.positions:
            if position['Status'] == 0:  # Only open positions
                current_price = self.data.iloc[self.current_step]["close"]
                entry_price = position['ActionPrice']

                if position['Type'] == 'BUY':
                    unrealized_pnl = (current_price - entry_price) * self.point
                else:  # Sell
                    unrealized_pnl = (entry_price - current_price) * self.point

                total_equity += unrealized_pnl

        return total_equity

    def render(self, mode='human', title=None, **kwargs):
        # Render the environment to the screen
        if mode in ('human', 'file'):
            log_header      =   True
            printout        =   False
            if mode == 'human':
                printout    =   True

            log_file = self.csv_file.replace("split/", "log/")
            pm = {
                "log_header": log_header,
                "log_filename": log_file,
                "printout": printout,
                "balance": self.balance,
                "balance_initial": self.balance_initial,
                "transaction_close_this_step": self.positions,
                "done_information": False
            }
            render_to_file(**pm)
            if log_header:
                    log_header = False